In [1]:
%matplotlib inline

import gym
import matplotlib
import numpy as np
import sys

from collections import defaultdict
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.blackjack import BlackjackEnv
from lib import plotting

matplotlib.style.use('ggplot')

In [2]:
env = BlackjackEnv()

In [6]:
def make_epsilon_greedy_policy(Q, epsilon, nA):
    """
    Creates an epsilon-greedy policy based on a given Q-function and epsilon.
    
    Args:
        Q: A ensted dictionary that maps from state -> (action -> action-value).
            The action values are of type float.
        epsilon: The probability to select a random action with. Float between 0 and 1.
        nA: Number of actions to pick from.
    
    Returns:
        A function that takes the observation as an argument and returns an action.
    
    """
    def policy_fn(observation):
        # Take a random action with probability epsilon
        if np.random.rand() <= epsilon or len(Q[observation]) == 0: 
            return np.random.randint(nA)
        # Take the best possible action according to the action-value function
        else:
            possible_actions = Q[observation]
            best_action, best_action_value = max(possible_actions.items(), key=lambda x: x[1])
            return best_action
    return policy_fn

In [7]:
def mc_control_epsilon_greedy(env, num_episodes, discount_factor=1.0, epsilon=0.1):
    """
    Monte Carlo Control using Epsilon-Greedy policies.
    Finds an optimal epsilon-greedy policy.
    
    Args:
        env: OpenAI gym environment.
        num_episodes: Nubmer of episodes to sample.
        discount_factor: Lambda discount factor.
        epsilon: Chance the sample a random action. Float betwen 0 and 1.
    
    Returns:
        A tuple (Q, policy).
        Q is a nested dictionary mapping state -> (action -> action-value).
        policy is a function taht takes an observation as an argument and returns an action.
    """
    
    # Keeps track of sum and count of returns for each state
    # to calculate an average. We could use an array to save all
    # returns (like in the book) but that's memory inefficient.
    returns_sum = defaultdict(float)
    returns_count = defaultdict(float)
    
    # The final action-value function.
    # A nested dictionary that maps state -> (action -> action-value).
    Q = defaultdict(lambda: defaultdict(float))
    
    # The policy we're following
    policy = make_epsilon_greedy_policy(Q, epsilon, env.action_space.n)
    
    # Implement!
        
    return Q, policy

In [8]:
Q, policy = mc_control_epsilon_greedy(env, num_episodes=1000000, epsilon=0.1)

In [9]:
# For plotting: Create value function from action-value function
# by picking the best action at each state
V = defaultdict(float)
for state, actions in Q.items():
    best_action, action_value = max(actions.items(), key=lambda x: x[1])
    V[state] = action_value
plotting.plot_value_function(V, title="Optimal Value Function")

ValueError: min() arg is an empty sequence